In [31]:
!pip install kfp==1.8.9
!pip install google-cloud-pipeline-components==0.2.0

In [1]:
import kfp
from kfp import components
import kfp.dsl as dsl
from kfp.v2 import compiler
from kfp.v2.google.client import AIPlatformClient
from google_cloud_pipeline_components import aiplatform as gcc_aip

from pipelines.deploy_model_pipeline import deploy_model_xgb_pipeline

In [2]:
compiler.Compiler().compile(
    pipeline_func=deploy_model_xgb_pipeline,
    package_path="dm-pl.json"
)


/opt/conda/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1266: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


In [3]:
from google.cloud.aiplatform.pipeline_jobs import PipelineJob

pl = PipelineJob(display_name= 'model-deploy-xgb-job',
        template_path= "dm-pl.json",
        location='us-central1',
        pipeline_root="gs://feature-store-mars21/vertex-xgb-ml-ops/a3b8252",
        parameter_values={'project_id': 'feature-store-mars21', 
                          'region': 'us-central1',
                          'metadata_name': 'default',
                          'model_uri_path': 'gs://feature-store-mars21/vertex-xgb-ml-ops'})

pl.run(sync=False)

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob


In [45]:

from google.cloud import aiplatform as aiplatform
df = aiplatform.get_pipeline_df(pipeline="xgb-pipeline")

In [46]:
df

,pipeline_name,run_name,param.input:endpoint_name,param.input:data_path,param.input:project_id,metric.auc,metric.confusionMatrix
0,xgb-pipeline,build-eb3b19f,xgb-ep,gs://mortgage_dataset_files/mortgage-small.csv,feature-store-mars21,0.970745,"{'rows': [{'row': [28.0, 0.0]}, {'row': [0.0, ..."
1,xgb-pipeline,xgb-pipeline-20211202194851,xgb-ep,gs://mortgage_dataset_files/mortgage-small.csv,feature-store-mars21,0.982732,"{'rows': [{'row': [26.0, 0.0]}, {'row': [0.0, ..."
2,xgb-pipeline,schedule-run-a3b8252-20211126t120000625z,xgb-ep,gs://mortgage_dataset_files/mortgage-small.csv,feature-store-mars21,0.982732,"{'rows': [{'row': [26.0, 0.0]}, {'row': [0.0, ..."
3,xgb-pipeline,schedule-run-a3b8252-20211126t090000554z,xgb-ep,gs://mortgage_dataset_files/mortgage-small.csv,feature-store-mars21,0.982732,"{'rows': [{'row': [26.0, 0.0]}, {'row': [0.0, ..."
4,xgb-pipeline,schedule-run-a3b8252-20211126t060000494z,xgb-ep,gs://mortgage_dataset_files/mortgage-small.csv,feature-store-mars21,0.982732,"{'annotationSpecs': [{'displayName': '0'}, {'d..."
...,...,...,...,...,...,...,...
105,xgb-pipeline,my-test-r7,xgb-ep,gs://mortgage_dataset_files/mortgage-small.csv,feature-store-mars21,0.975667,"{'rows': [{'row': [26.0, 0.0]}, {'row': [0.0, ..."
106,xgb-pipeline,my-test-r6,xgb-ep,gs://mortgage_dataset_files/mortgage-small.csv,feature-store-mars21,0.975667,"{'annotationSpecs': [{'displayName': '0'}, {'d..."
107,xgb-pipeline,my-test-r5,xgb-ep,gs://mortgage_dataset_files/mortgage-small.csv,feature-store-mars21,0.975667,"{'annotationSpecs': [{'displayName': '0'}, {'d..."
108,xgb-pipeline,my-test-r4,xgb-ep,gs://mortgage_dataset_files/mortgage-small.csv,feature-store-mars21,0.975667,"{'annotationSpecs': [{'displayName': '0'}, {'d..."


In [6]:
from google.cloud import aiplatform_v1 

PROJECT_ID='feature-store-mars21'
REGION='us-central1'

In [7]:
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
metadata_client = aiplatform_v1.MetadataServiceClient(
  client_options={
      "api_endpoint": API_ENDPOINT
  }
)

In [39]:
import pandas as pd
MODEL_FILTER="schema_title = \"system.Model\""
MODEL_FILTER="uri = \"gs://feature-store-mars21/vertex-xgb-ml-ops/*\" AND schema_title=\"system.Model\""
artifact_request = aiplatform_v1.ListArtifactsRequest(
    parent="projects/{0}/locations/{1}/metadataStores/default".format(PROJECT_ID, REGION),
    filter=MODEL_FILTER
)
model_artifacts = metadata_client.list_artifacts(artifact_request)

time_last=None
uri_last =None
for ma in model_artifacts:
    if uri_last==None:
        uri_last=ma.uri
        time_last=ma.create_time
    elif(time_last<ma.create_time):
        uri_last=ma.uri
        time_last=ma.create_time
 

        

uri_last

'gs://feature-store-mars21/vertex-xgb-ml-ops/d487c43/202835066335/build-vertex-xgb-ml-ops-d487c43/xgb-training_-6195875565866582016/model'

In [32]:
df

,uri,createTime
0,gs://feature-store-mars21/vertex-xgb-ml-ops/d4...,2021-12-02 23:09:25.600000+00:00
1,gs://feature-store-mars21/vertex-xgb-ml-ops/27...,2021-12-02 22:37:44.914000+00:00
2,gs://feature-store-mars21/vertex-xgb-ml-ops/27...,2021-12-02 22:27:15.599000+00:00
3,gs://feature-store-mars21/vertex-xgb-ml-ops/27...,2021-12-02 22:11:40.487000+00:00
4,gs://feature-store-mars21/vertex-xgb-ml-ops/86...,2021-12-02 22:10:24.346000+00:00
5,gs://feature-store-mars21/vertex-xgb-ml-ops/8e...,2021-12-02 22:04:26.772000+00:00
6,gs://feature-store-mars21/vertex-xgb-ml-ops/a3...,2021-12-02 21:56:45.815000+00:00
7,gs://feature-store-mars21/vertex-xgb-ml-ops/27...,2021-12-02 21:37:03.756000+00:00
8,gs://feature-store-mars21/vertex-xgb-ml-ops/a3...,2021-12-02 21:04:38.610000+00:00
9,gs://feature-store-mars21/vertex-xgb-ml-ops/a3...,2021-12-02 21:04:36.544000+00:00


In [14]:
PL_FILTER="display-name = \"sr-vertex-xgb-ml-ops-\""
context_request = aiplatform_v1.ListContextsRequest(
    parent="projects/{0}/locations/{1}/metadataStores/default".format(PROJECT_ID, REGION),
    filter=PL_FILTER)

model_artifacts = metadata_client.list_contexts(context_request)
model_artifacts

InvalidArgument: 400 Invalid field: display-name provided in comparison expression

In [10]:
PL_FILTER=""
context_request = aiplatform_v1.ListExecutionsRequest(
    parent="projects/{0}/locations/{1}/metadataStores/default".format(PROJECT_ID, REGION),
    filter=PL_FILTER)

model_artifacts = metadata_client.list_executions(context_request)
model_artifacts

ListExecutionsPager<executions {
  name: "projects/202835066335/locations/us-central1/metadataStores/default/executions/2094499329187972768"
  display_name: "split-data"
  state: RUNNING
  etag: "1638486431168"
  create_time {
    seconds: 1638486430
    nanos: 798000000
  }
  update_time {
    seconds: 1638486431
    nanos: 168000000
  }
  schema_title: "system.ContainerExecution"
  schema_version: "0.0.1"
  metadata {
    fields {
      key: "input:data_path"
      value {
        string_value: "gs://feature-store-mars21/mortgage_dataset_files/mortgage-small.csv"
      }
    }
  }
}
executions {
  name: "projects/202835066335/locations/us-central1/metadataStores/default/executions/6251223551884359176"
  display_name: "build-vertex-xgb-ml-ops-d487c43"
  state: RUNNING
  etag: "1638486430371"
  create_time {
    seconds: 1638486430
    nanos: 371000000
  }
  update_time {
    seconds: 1638486430
    nanos: 371000000
  }
  schema_title: "system.Run"
  schema_version: "0.0.1"
  metadata 

In [6]:
model_uri_path="test/path"


if not model_uri_path.endswith("/"):
        model_uri_path+='/'

In [4]:

from google_cloud_pipeline_components import aiplatform as gcc_aip


In [ ]:
gcc_aip.moin